# 01 — Data Collection (Netflix App Reviews)

This notebook collects Netflix reviews from:
- **Google Play Store**
- **Apple App Store**


✅ Output of this notebook:  
`../data/netflix_reviews_raw.csv`

This file will be used in **02_data_cleaning.ipynb**.


## Step 1: Imports & Setup

In [ ]:

import requests
import pandas as pd
from pathlib import Path

In [ ]:
from google_play_scraper import reviews as play_reviews, Sort


In [ ]:
# 1. Scrape Google Play Store Reviews (multi-country)
# ---------------------------
def get_playstore_reviews(app_id, countries, max_reviews=10000):
    all_reviews = []
    for country in countries:
        print(f"\nScraping Play Store reviews ({country})...")
        token = None
        country_reviews = []
        while True:
            reviews, token = play_reviews(
                app_id,
                lang='en',
                country=country.lower(),
                sort=Sort.NEWEST,
                count=200,
                continuation_token=token
            )
            for r in reviews:
                r["country"] = country.upper()
            country_reviews.extend(reviews)
            print(f"Collected {len(country_reviews)} Play Store reviews for {country} so far...")

            if not token or len(country_reviews) >= max_reviews:
                break
        all_reviews.extend(country_reviews)
    return all_reviews

In [ ]:
# 2. Scrape Apple App Store Reviews (multi-country)
# ---------------------------
def get_appstore_reviews(app_id, country="us", pages=10):
    """Scrape up to ~50 reviews per page. For 500 reviews, use ~10 pages."""
    all_reviews = []
    for page in range(1, pages + 1):
        url = f"https://itunes.apple.com/{country}/rss/customerreviews/id={app_id}/page={page}/json"
        r = requests.get(url)
        if r.status_code != 200:
            print(f"Error fetching page {page} for {country}: {r.status_code}")
            break

        data = r.json()
        if "feed" not in data or "entry" not in data["feed"]:
            print(f"No more reviews found on page {page} for {country}")
            break

        entries = data["feed"]["entry"]

        # First entry is the app itself
        for review in entries[1:]:
            all_reviews.append({
                "username": review["author"]["name"]["label"],
                "review": review["content"]["label"],
                "rating": int(review["im:rating"]["label"]),
                "date": review["updated"]["label"],
                "country": country.upper()
            })

        print(f"Fetched {len(all_reviews)} reviews so far for {country}...")

        if len(all_reviews) >= 500:  # stop after 500
            break

    return all_reviews



## Step 2: Fetch Reviews
We run the scraping functions to retrieve reviews from both platforms.


In [ ]:
# 3. Run Scrapers
# ---------------------------
play_countries = ["NG", "CA", "US", "GB"]
app_countries = ["ng", "ca", "us", "gb"]
app_id = 363590051  # Netflix App Store ID


In [ ]:
# 4. Play Store
play_reviews_all = get_playstore_reviews("com.netflix.mediaclient", play_countries, max_reviews=10000)
play_df = pd.DataFrame(play_reviews_all)
play_df = play_df[['userName', 'content', 'score', 'at', 'country']]
play_df.rename(columns={'userName': 'username', 'content': 'review',
                        'score': 'rating', 'at': 'date'}, inplace=True)
play_df['source'] = 'Play Store'

In [ ]:
# 5. App Store
all_appstore_reviews = []
for c in app_countries:
    print(f"\nScraping App Store reviews ({c.upper()})...")
    reviews = get_appstore_reviews(app_id, country=c, pages=10)
    all_appstore_reviews.extend(reviews)

app_df = pd.DataFrame(all_appstore_reviews)
app_df['source'] = 'App Store'

## Step 3: Combine Sources into a Single Dataset


In [ ]:
# Combine both outcome
combined_df = pd.concat([play_df, app_df], ignore_index=True)
print(f"✅ Total combined reviews: {len(combined_df)}")
combined_df

## Step 4: Save to `data/`


In [ ]:
# To CSV
RAW_DATA_PATH = Path("../data/netflix_reviews_raw.csv")

In [ ]:
# Save dataset
combined_df.to_csv(RAW_DATA_PATH, index=False)

print("\n✅ Saved raw combined dataset to 'data/netflix_reviews_raw.csv'")
print(f"📊 Total entries: {len(combined_df)}")

In [ ]:
# Combined Dataset Summary
# After merging Play Store and App Store reviews, we save the combined dataset and show the counts for each source.

print("\n✅ Saved raw combined dataset to 'data/netflix_reviews_raw.csv'")
print(f"Play Store reviews: {len(play_df)}")
print(f"App Store reviews: {len(app_df)}")
print(f"Total combined: {len(combined_df)}")

### Data Collection Completed!

| Source          | Reviews Collected |
|----------------|------------------|
| Play Store     | ✅ `{len(play_df)}` |
| App Store      | ✅ `{len(app_df)}` |
| **Total Saved** | 🎯 `{len(combined_df)}` |

➡️ Next Notebook: **02_data_cleaning.ipynb**
